In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain
from lightgbm import LGBMRegressor
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from xgboost import XGBRegressor



from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import StackingRegressor

import matplotlib.pyplot as plt

from scipy import stats
import numpy as np
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

/Users/donaldaadithiyan/Personal learn dev/YT-analytics/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
preprocessed_dataset = pd.read_csv("../data/pre-processed/preprocessed_youtube_data.csv")

In [3]:
# Convert to datetime if not already
preprocessed_dataset['published_time'] = pd.to_datetime(preprocessed_dataset['published_time'], format='%H:%M:%S', errors='coerce')

# Extract hour and minute
preprocessed_dataset['published_hour'] = preprocessed_dataset['published_time'].dt.hour
preprocessed_dataset['published_minute'] = preprocessed_dataset['published_time'].dt.minute

# Drop the original time column
preprocessed_dataset.drop(columns=['published_time'], inplace=True)

In [4]:
# Separate label encoders for each column
le_day = LabelEncoder()
le_def = LabelEncoder()

preprocessed_dataset['published_day_of_week'] = le_day.fit_transform(preprocessed_dataset['published_day_of_week'])
preprocessed_dataset['definition'] = le_def.fit_transform(preprocessed_dataset['definition'])


In [5]:
missing_mask = preprocessed_dataset[['like_count_initial', 'like_count_final', 'view_count_initial', 'view_count_final']].isnull()

# Check rows where all four columns are NaN
rows_all_nan = missing_mask.all(axis=1)

print("Number of rows with all four columns NaN:", rows_all_nan.sum())

# Optionally, see those rows
print(preprocessed_dataset[rows_all_nan])

Number of rows with all four columns NaN: 15
       category_id  definition  view_count_initial  like_count_initial  \
360            NaN           2                 NaN                 NaN   
414            NaN           2                 NaN                 NaN   
1053           NaN           2                 NaN                 NaN   
1635           NaN           2                 NaN                 NaN   
3221           NaN           2                 NaN                 NaN   
4503           NaN           2                 NaN                 NaN   
8791           NaN           2                 NaN                 NaN   
13543          NaN           2                 NaN                 NaN   
16293          NaN           2                 NaN                 NaN   
17041          NaN           2                 NaN                 NaN   
17227          NaN           2                 NaN                 NaN   
17656          NaN           2                 NaN                 

In [6]:
rows_any_nan = missing_mask.any(axis=1)
print("Rows with any of the four columns NaN:", rows_any_nan.sum())

Rows with any of the four columns NaN: 728


In [7]:
preprocessed_dataset = preprocessed_dataset.dropna(subset=[
    'like_count_initial',
    'like_count_final',
    'view_count_initial',
    'view_count_final'
])

# Define target columns
target_columns = [
    'like_count_initial',
    'like_count_final',
    'view_count_initial',
    'view_count_final'
]

In [8]:
# Choose columns to check for outliers, e.g. target columns
cols_to_check = target_columns  # or other numeric features

z_scores = np.abs(stats.zscore(preprocessed_dataset[cols_to_check]))
# Define threshold, e.g. 3 std deviations
threshold = 3
# Keep only rows where all z-scores are below threshold (no outlier)
non_outliers = (z_scores < threshold).all(axis=1)


print(f"Rows before outlier removal: {len(preprocessed_dataset)}")
preprocessed_dataset = preprocessed_dataset[non_outliers]
print(f"Rows after outlier removal: {len(preprocessed_dataset)}")

Rows before outlier removal: 20613
Rows after outlier removal: 20399


In [9]:
# Separate features and targets
X = preprocessed_dataset.drop(columns=target_columns)
y = preprocessed_dataset[target_columns]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
# 2. Define objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
        "random_state": 42,
        "n_jobs": -1,
    }

    xgb = XGBRegressor(**params)
    model = MultiOutputRegressor(xgb)
    model.fit(X_train, y_train)

    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 3. Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

# 4. Train final model with best params
best_params = study.best_trial.params
best_model = MultiOutputRegressor(XGBRegressor(**best_params))
best_model.fit(X_train, y_train)

# 5. Predict and evaluate
y_pred = best_model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)
print(f"✅ Best MAE: {mae:.4f}")

[I 2025-07-31 09:14:14,541] A new study created in memory with name: no-name-6c1985e7-84d7-4ac3-aebf-6880ead1c11c
Best trial: 0. Best value: 334.798:   3%|▎         | 1/30 [00:01<00:36,  1.27s/it]

[I 2025-07-31 09:14:15,817] Trial 0 finished with value: 334.797607421875 and parameters: {'n_estimators': 221, 'max_depth': 4, 'learning_rate': 0.01360403206523462, 'subsample': 0.9381384114391893, 'colsample_bytree': 0.5994871129710978, 'gamma': 2.0872107138738807, 'reg_alpha': 1.1939013552736917, 'reg_lambda': 1.6419752421957785}. Best is trial 0 with value: 334.797607421875.


Best trial: 0. Best value: 334.798:   7%|▋         | 2/30 [00:02<00:32,  1.15s/it]

[I 2025-07-31 09:14:16,877] Trial 1 finished with value: 352.4061279296875 and parameters: {'n_estimators': 238, 'max_depth': 3, 'learning_rate': 0.2349679240967145, 'subsample': 0.6907801228244059, 'colsample_bytree': 0.5458908080791052, 'gamma': 3.4441800793754185, 'reg_alpha': 2.0081120668773007, 'reg_lambda': 0.11520751158410947}. Best is trial 0 with value: 334.797607421875.


Best trial: 0. Best value: 334.798:  10%|█         | 3/30 [00:03<00:29,  1.08s/it]

...

[I 2025-07-31 09:16:06,649] Trial 27 finished with value: 307.5177001953125 and parameters: {'n_estimators': 397, 'max_depth': 7, 'learning_rate': 0.014760642256955907, 'subsample': 0.7368063841659224, 'colsample_bytree': 0.7366794751976453, 'gamma': 0.9609372610906219, 'reg_alpha': 1.5877862510589986, 'reg_lambda': 0.5305203665253693}. Best is trial 10 with value: 302.282958984375.


Best trial: 10. Best value: 302.283:  97%|█████████▋| 29/30 [01:54<00:03,  3.15s/it]

[I 2025-07-31 09:16:08,587] Trial 28 finished with value: 314.23406982421875 and parameters: {'n_estimators': 296, 'max_depth': 5, 'learning_rate': 0.05257197585097488, 'subsample': 0.6537160224887318, 'colsample_bytree': 0.8168586718503508, 'gamma': 0.3950809179303537, 'reg_alpha': 2.550851754759975, 'reg_lambda': 0.7938942603604289}. Best is trial 10 with value: 302.282958984375.


Best trial: 10. Best value: 302.283: 100%|██████████| 30/30 [01:57<00:00,  3.91s/it]


[I 2025-07-31 09:16:11,890] Trial 29 finished with value: 308.6338806152344 and parameters: {'n_estimators': 614, 'max_depth': 4, 'learning_rate': 0.011157343135218739, 'subsample': 0.9801067903015792, 'colsample_bytree': 0.958776319588503, 'gamma': 0.07589961420986298, 'reg_alpha': 0.009881011262728334, 'reg_lambda': 1.403606257420083}. Best is trial 10 with value: 302.282958984375.
✅ Best MAE: 302.6985


In [11]:
# Convert to DataFrame for easier handling if needed
if not isinstance(y_valid, pd.DataFrame):
    y_valid = pd.DataFrame(y_valid, columns=target_columns)
if not isinstance(y_pred, pd.DataFrame):
    y_pred = pd.DataFrame(y_pred, columns=target_columns)

# MAE per target column
mae_per_column = {}
for col in target_columns:
    mae_per_column[col] = mean_absolute_error(y_valid[col], y_pred[col])

print("MAE per target column:")
for col, val in mae_per_column.items():
    print(f"{col}: {val:.4f}")

MAE per target column:
like_count_initial: 19.7055
like_count_final: 20.7099
view_count_initial: 566.3317
view_count_final: 604.0472


In [12]:
def catboost_objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "random_strength": trial.suggest_float("random_strength", 1, 20),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_seed": 42,
        "verbose": False,
    }

    catBoost = CatBoostRegressor(**params)
    model = MultiOutputRegressor(catBoost)
    model.fit(X_train, y_train)

    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 3. Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(catboost_objective, n_trials=30, show_progress_bar=True)

# 4. Train final model with best params
best_params = study.best_trial.params
best_model = MultiOutputRegressor(CatBoostRegressor(**best_params))
best_model.fit(X_train, y_train)

# 5. Predict and evaluate
y_pred = best_model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)

# Convert y_valid and y_pred to DataFrames if they aren't already, using target column names
if not isinstance(y_valid, pd.DataFrame):
    y_valid = pd.DataFrame(y_valid, columns=target_columns)
if not isinstance(y_pred, pd.DataFrame):
    y_pred = pd.DataFrame(y_pred, columns=target_columns)

# Calculate MAE for each column
print(f"✅ Best Overall MAE: {mae:.4f}\n")
print("MAE per target column:")
for col in target_columns:
    col_mae = mean_absolute_error(y_valid[col], y_pred[col])
    print(f"{col}: {col_mae:.4f}")

[I 2025-07-31 09:16:14,652] A new study created in memory with name: no-name-e64cc4a8-b07f-4351-9031-b11679b466e9
Best trial: 0. Best value: 327.331:   3%|▎         | 1/30 [00:02<01:04,  2.23s/it]

[I 2025-07-31 09:16:16,884] Trial 0 finished with value: 327.33071628934397 and parameters: {'iterations': 478, 'depth': 5, 'learning_rate': 0.1983252933628219, 'random_strength': 2.078755107096994, 'bagging_temperature': 0.17770625988323896, 'l2_leaf_reg': 4.457169365920551, 'border_count': 213}. Best is trial 0 with value: 327.33071628934397.


Best trial: 0. Best value: 327.331:   7%|▋         | 2/30 [00:15<04:01,  8.61s/it]

[I 2025-07-31 09:16:29,962] Trial 1 finished with value: 338.7821945777454 and parameters: {'iterations': 365, 'depth': 12, 'learning_rate': 0.23219305196267334, 'random_strength': 5.813905077323028, 'bagging_temperature': 0.2720254280200962, 'l2_leaf_reg': 3.5113368360789434, 'border_count': 120}. Best is trial 0 with value: 327.33071628934397.


Best trial: 0. Best value: 327.331:  10%|█         | 3/30 [00:18<02:50,  6.32s/it]

...

[I 2025-07-31 09:18:15,353] Trial 27 finished with value: 319.95540154035945 and parameters: {'iterations': 673, 'depth': 10, 'learning_rate': 0.09882765967902381, 'random_strength': 12.641776930093817, 'bagging_temperature': 0.5136205004273531, 'l2_leaf_reg': 1.0548544770640196, 'border_count': 207}. Best is trial 22 with value: 304.3577414444444.


Best trial: 22. Best value: 304.358:  97%|█████████▋| 29/30 [02:06<00:06,  6.21s/it]

[I 2025-07-31 09:18:20,965] Trial 28 finished with value: 322.4647062771293 and parameters: {'iterations': 309, 'depth': 10, 'learning_rate': 0.14962523461414856, 'random_strength': 3.792385246943592, 'bagging_temperature': 0.018966674524985128, 'l2_leaf_reg': 1.9785684466270266, 'border_count': 223}. Best is trial 22 with value: 304.3577414444444.


Best trial: 22. Best value: 304.358: 100%|██████████| 30/30 [02:08<00:00,  4.27s/it]


[I 2025-07-31 09:18:22,720] Trial 29 finished with value: 315.2159060109156 and parameters: {'iterations': 428, 'depth': 4, 'learning_rate': 0.046129613637102766, 'random_strength': 18.521320633977133, 'bagging_temperature': 0.17286554310480795, 'l2_leaf_reg': 3.95026127716016, 'border_count': 207}. Best is trial 22 with value: 304.3577414444444.
0:	learn: 146.0225257	total: 1.5ms	remaining: 630ms
1:	learn: 144.4744168	total: 2.89ms	remaining: 606ms
2:	learn: 142.4908743	total: 4.34ms	remaining: 606ms
3:	learn: 140.5695205	total: 5.71ms	remaining: 596ms
4:	learn: 138.8963530	total: 7.03ms	remaining: 586ms
5:	learn: 137.0228174	total: 8.56ms	remaining: 593ms
6:	learn: 135.4956390	total: 10.1ms	remaining: 597ms
7:	learn: 134.2974527	total: 11.9ms	remaining: 614ms
8:	learn: 132.5611585	total: 13.3ms	remaining: 609ms
9:	learn: 131.6397477	total: 14.8ms	remaining: 611ms
10:	learn: 130.0718625	total: 16.7ms	remaining: 624ms
11:	learn: 128.8980751	total: 18.4ms	remaining: 627ms
12:	learn: 127

In [13]:
def lightGBM_objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 20),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
    }


    lightGBM = LGBMRegressor(**params)
    model = MultiOutputRegressor(lightGBM)
    model.fit(X_train, y_train)

    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# 3. Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(lightGBM_objective, n_trials=30, show_progress_bar=True)

# 4. Train final model with best params
best_params = study.best_trial.params
best_model = MultiOutputRegressor(LGBMRegressor(**best_params))
best_model.fit(X_train, y_train)

# 5. Predict and evaluate
y_pred = best_model.predict(X_valid)
mae = mean_absolute_error(y_valid, y_pred)

# Convert y_valid and y_pred to DataFrames if they aren't already, using target column names
if not isinstance(y_valid, pd.DataFrame):
    y_valid = pd.DataFrame(y_valid, columns=target_columns)
if not isinstance(y_pred, pd.DataFrame):
    y_pred = pd.DataFrame(y_pred, columns=target_columns)

# Calculate MAE for each column
print(f"✅ Best Overall MAE: {mae:.4f}\n")
print("MAE per target column:")
for col in target_columns:
    col_mae = mean_absolute_error(y_valid[col], y_pred[col])
    print(f"{col}: {col_mae:.4f}")

[I 2025-07-31 09:18:25,743] A new study created in memory with name: no-name-c43c9d8c-ff83-47f5-9b92-e80755b3f44e
  0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000555 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1721
[LightGBM] [Info] Number of data points in the train set: 16319, number of used features: 15
[LightGBM] [Info] Start training from score 33.251241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

Best trial: 0. Best value: 359.251:   3%|▎         | 1/30 [00:10<04:53, 10.11s/it]

[I 2025-07-31 09:18:35,851] Trial 0 finished with value: 359.25050269292444 and parameters: {'n_estimators': 295, 'learning_rate': 0.1678616438610672, 'max_depth': 9, 'num_leaves': 65, 'min_child_samples': 6, 'min_split_gain': 0.2720831319373088, 'subsample': 0.6726340109520282, 'colsample_bytree': 0.7589327856715941}. Best is trial 0 with value: 359.25050269292444.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000489 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1721
[LightGBM] [Info] Number of data points in the train set: 16319, number of used features: 15
[LightGBM] [Info] Start training from score 33.251241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

Best trial: 1. Best value: 316.441:   7%|▋         | 2/30 [00:13<02:50,  6.08s/it]

...

Best trial: 23. Best value: 300.952:  97%|█████████▋| 29/30 [02:01<00:03,  3.37s/it]

[I 2025-07-31 09:20:27,682] Trial 28 finished with value: 308.1124497232363 and parameters: {'n_estimators': 248, 'learning_rate': 0.040898998676766996, 'max_depth': 7, 'num_leaves': 56, 'min_child_samples': 19, 'min_split_gain': 0.13627596655035049, 'subsample': 0.8018755800277507, 'colsample_bytree': 0.974367568300534}. Best is trial 23 with value: 300.9522916936402.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1721
[LightGBM] [Info] Number of data points in the train set: 16319, number of used features: 15
[LightGBM] [Info] Start training from score 33.251241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

Best trial: 23. Best value: 300.952: 100%|██████████| 30/30 [02:07<00:00,  4.24s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[I 2025-07-31 09:20:32,819] Trial 29 finished with value: 327.126679153145 and parameters: {'n_estimators': 222, 'learning_rate': 0.08813741267213765, 'max_depth': 9, 'num_leaves': 66, 'min_child_samples': 17, 'min_split_gain': 0.22678523729070554, 'subsample': 0.6948728020032814, 'colsample_bytree': 0.8958396148002072}. Best is trial 23 with value: 300.9522916936402.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f